In [21]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

In [22]:
bart_model = LitModel.load_from_checkpoint("/content/drive/MyDrive/bart_model/output.ckpt",learning_rate = 2e-5, tokenizer = tokenizer, model = model).to("cuda")

In [23]:
def get_req_len_dict(dataset, istrain):
    '''
    function to required length data for each summary
    '''
    f = open('/content/drive/MyDrive/IN_Abs/test-data/stats-IN-test.txt', "r")
    a = (f.read())
    a = a.split("\n")
    dict_names = {}
    for i in a:
        b = i.split("	")
        try:
            tp = int(b[2])
            dict_names[b[0]] = tp
        except:
            print(b)
    return dict_names


def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)
    return sents

In [24]:
#Reading the test documents
names, data_source, data_summary = get_summary_data(dataset, "test")
print(len(names))
print(len(data_source))
print(len(data_summary))
dict_names = get_req_len_dict(dataset, "test")

100
100
100
['']


In [25]:
def nest_sentences(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a string.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(" ".join(sent))
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(" ".join(sent))
    return nested

In [26]:
def generate_summary_gpu(nested_sentences,p=0.2):
  '''
    Function to generate summaries from the list containing chunks of the document
    input:  nested_sentences - chunks
            p - Number of words in summaries per word in the document
    output: document summary
    '''
  device = 'cuda'
  summaries = []
  for nested in nested_sentences:
    l = int(p * len(nested.split(" ")))
    input_tokenized = tokenizer.encode(nested, truncation=True, return_tensors='pt')
    input_tokenized = input_tokenized.to(device)
    summary_ids = bart_model.model.to(device).generate(input_tokenized,
                                      length_penalty=0.01,
                                      min_length=l-5,
                                      max_length=l+5)
    output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

In [27]:
import os
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:

# main loop to generate and save summaries of each document in the test dataset
for i in range(len(data_source)):
    name = names[i]
    doc = data_source[i]
    wc = doc.split(" ")
    input_len = len(wc)
    req_len = dict_names[name]
    print(str(i) + ": " + name +  " - " + str(input_len) + " : " + str(req_len), end = " ,")

    nested = nest_sentences(doc,1024)
    l = int(req_len/len(nested))
    p = float(req_len/input_len)
    print(p)

    abs_summ = generate_summary_gpu(nested,p)
#     print(abs_summ)
#     break
    abs_summ = " ".join(abs_summ)
    if len(abs_summ.split(" ")) > req_len:
        abs_summ = abs_summ.split(" ")
        abs_summ = abs_summ[:req_len]
        abs_summ = " ".join(abs_summ)
#     print(abs_summ)
#     break
    print(len((abs_summ.split(" "))))

    path = output_path + name
    file = open(path,'w')
    file.write(abs_summ)
    file.close()

0: 1974.txt - 2686 : 529 ,0.19694713328369323
412
1: 1181.txt - 3387 : 321 ,0.09477413640389726
277
2: 1195.txt - 4234 : 531 ,0.12541332073689182
430
3: 1531.txt - 3038 : 514 ,0.16919025674786042
426
4: 1762.txt - 4819 : 552 ,0.11454658642871965
470
5: 1697.txt - 12399 : 1389 ,0.11202516331962255
1173
6: 1522.txt - 3109 : 272 ,0.0874879382438083
241
7: 1378.txt - 2202 : 205 ,0.09309718437783833
177
8: 1406.txt - 2089 : 410 ,0.19626615605552897
349
9: 1789.txt - 2076 : 346 ,0.16666666666666666
306
10: 1778.txt - 1463 : 468 ,0.3198906356801094
397
11: 1329.txt - 2990 : 343 ,0.11471571906354515
316
12: 362.txt - 6285 : 1038 ,0.16515513126491646
901
13: 3602.txt - 1978 : 421 ,0.2128412537917088
365
14: 3292.txt - 1611 : 577 ,0.3581626319056487
458
15: 314.txt - 2829 : 258 ,0.0911983032873807
232
16: 3168.txt - 3174 : 736 ,0.2318840579710145
644
17: 3210.txt - 3650 : 603 ,0.1652054794520548
529
18: 2593.txt - 2755 : 630 ,0.22867513611615245
529
19: 2627.txt - 2820 : 752 ,0.26666666666666666

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=89a9cd60a6a59072499b06a9173acfef768c3f307189a29d108e09253212ac04
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import os
import pandas as pd
from rouge_score import rouge_scorer

# Define the paths to the folders containing documents

folder1_path = "/content/drive/MyDrive/pred_summ"
folder2_path = "/content/drive/MyDrive/IN_Abs/test-data/summary"

# Get the list of files in each folder

folder1_files = sorted(os.listdir(folder1_path))
folder2_files = sorted(os.listdir(folder2_path))

# Initialize lists to store filenames and ROUGE scores
file_pairs = []
rouge_scores = []

# Iterate over the files in both folders
for file1, file2 in zip(folder1_files, folder2_files):
    # Read the content from the files
    with open(os.path.join(folder1_path, file1), "r", encoding="utf-8") as file:
        content1 = file.read()
    with open(os.path.join(folder2_path, file2), "r", encoding="utf-8") as file:
        content2 = file.read()

    # Calculate ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(content1, content2)

    # Append filename pair and ROUGE scores to lists
    file_pairs.append((file1, file2))
    rouge_scores.append(scores)

# Create DataFrame from the lists
df = pd.DataFrame({
    'File Pair': file_pairs,
    'ROUGE-1 F1': [score['rouge1'].fmeasure for score in rouge_scores],
    'ROUGE-2 F1': [score['rouge2'].fmeasure for score in rouge_scores],
    'ROUGE-L F1': [score['rougeL'].fmeasure for score in rouge_scores]
})

# Save DataFrame to Excel file
df.to_excel("rouge_scores.xlsx", index=False)


In [ ]:
df

,File Pair,ROUGE-1 F1,ROUGE-2 F1,ROUGE-L F1
0,"(1181.txt, 1181.txt)",0.520000,0.193980,0.273333
1,"(1195.txt, 1195.txt)",0.567623,0.260780,0.252049
2,"(1329.txt, 1329.txt)",0.589474,0.295626,0.309774
3,"(1378.txt, 1378.txt)",0.532637,0.225722,0.240209
4,"(1406.txt, 1406.txt)",0.611979,0.292428,0.299479
...,...,...,...,...
95,"(7109.txt, 7109.txt)",0.676626,0.422535,0.418278
96,"(7130.txt, 7130.txt)",0.688436,0.377547,0.326546
97,"(715.txt, 715.txt)",0.563559,0.255319,0.292373
98,"(78.txt, 78.txt)",0.549598,0.244624,0.289544


In [ ]:

required_columns = ['ROUGE-1 F1',	'ROUGE-2 F1',	'ROUGE-L F1']


avg_values = df[required_columns].mean()

# Print the average values
print("Average values of the required columns:")
print(avg_values)

Average values of the required columns:
ROUGE-1 F1    0.628092
ROUGE-2 F1    0.340833
ROUGE-L F1    0.316778
dtype: float64


## *Taking 1 Documnet and Generating Summary*



In [44]:
name = names[0]
print("Document Name :-",name)
doc0 = data_source[0]
print(" ")
print("Document Data")
print(" ")
print(doc0)

Document Name :- 3210.txt
 
Document Data
 
Appeal No. 1604 of 1967.
From the judgment and order dated the 8th February, 1967 of Le Delhi High Court in Civil Writ No. 531 D of 1964.
P.P. Rao and R. N. Sachthey, for the appellants.
G.P. Pai, P. C. Bhartari and O. C. Mathur, for respondents nos.
1 & 2.
The Judgment of the Court was delivered by SARKARIA J.
Whether the Federation of Indian Chambers of Commerce and Industry, New Delhi (Respondent 1) is a "commercial establishment" within the meaning of s.2(5) of the Delhi Shops ,and Establishments Act, 1954 (for short the Act) is the sale question that falls for determination in this appeal by certificate directed against the judgment dated February 8, 1967 of the High Court of Delhi.
The facts bearing on this question may now be stated: Respondent 1 (hereinafter referred to as the Federation) is a Company registered under section 26 of the Indian Companies Act, 1913.
The primary objects of the Federation as given in the Memorandum of Asso

In [45]:
wc = doc0.split(" ")
input_len = len(wc)
req_len = dict_names[name]
print(str(i) + ": " + name +  " - " + str(input_len) + " : " + str(req_len), end = " ,")
nested = nest_sentences(doc0,1024)
abs_summ = generate_summary_gpu(nested,p)
abs_summ = " ".join(abs_summ)
if len(abs_summ.split(" ")) > req_len:
  abs_summ = abs_summ.split(" ")
  abs_summ = abs_summ[:req_len]
  abs_summ = " ".join(abs_summ)
  print("Summary Generated")
  print(" ")
  print(abs_summ)

1: 3210.txt - 3650 : 603 ,Summary Generated
 
 The [352, referred to.The primary objects of the Federation are to promote Indian business in matters of inland and foreign trade, transport, industry and manufactures, finance and all other economic subjects and to encourage Indian banking, shipping and insurance, to secure organised action on all subjects connected with the common good of Indian business, to collect and disseminate statistical and other information and to make effort for the spread of commercial and economic knowledge, to take all steps by lawful means which may be necessary for promoting supporting or opposing legislation or other action affecting the aforesaid economic interests and in general to take the initiative to assist and promote trade commerce and industry, to provide for arbitration in respect of disputes arising, in the course of trade, industry or transport or other business matters, and to secure the services of expert technical and other men to that end i

In [46]:
abs_summ

' The [352, referred to.The primary objects of the Federation are to promote Indian business in matters of inland and foreign trade, transport, industry and manufactures, finance and all other economic subjects and to encourage Indian banking, shipping and insurance, to secure organised action on all subjects connected with the common good of Indian business, to collect and disseminate statistical and other information and to make effort for the spread of commercial and economic knowledge, to take all steps by lawful means which may be necessary for promoting supporting or opposing legislation or other action affecting the aforesaid economic interests and in general to take the initiative to assist and promote trade commerce and industry, to provide for arbitration in respect of disputes arising, in the course of trade, industry or transport or other business matters, and to secure the services of expert technical and other men to that end if necessary or desirable, to conduct, underta